# Installing libraires

In [1]:
!pip install torch
!pip install opencv-python
!pip install matplotlib

# Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import itertools
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
%matplotlib inline

# Creating dataset

In [3]:
idendities = {
    "Angelina": ["img1.jpg", "img2.jpg", "img4.jpg", "img5.jpg", "img6.jpg", "img7.jpg", "img10.jpg", "img11.jpg"],
    "Scarlett": ["img8.jpg", "img9.jpg","img47.jpg"],
    "Jennifer": ["img3.jpg", "img12.jpg","img54.jpg"],
    "Mark": ["img13.jpg", "img14.jpg", "img15.jpg"],
    "Jack": ["img16.jpg", "img17.jpg"],
    "Elon": ["img18.jpg", "img19.jpg"],
    "Jeff": ["img20.jpg", "img21.jpg"],
    "Marissa": ["img22.jpg", "img23.jpg"],
    "Sundar": ["img24.jpg", "img25.jpg"]
}

## Training dataset

In [4]:
import random
# creating list anchor,positive, negative
positives = []
anchors = []
negatives = []
for key, values in idendities.items():
    if len(values) < 2:
        continue
    # Appending positive and anchor images
    for i in range(0, len(values)-1):
        for j in range(i+1, len(values)):
            positives.append(values[i])
            anchors.append(values[j])
    other_people = list(idendities.keys())
    other_people.remove(key)
    # Appending negative images
    for i in range(len(values)):
        neg_person = random.choice(other_people)
        neg_image = random.choice(idendities[neg_person])
        negatives.append(neg_image)

In [5]:
len(negatives)

27

In [6]:
data = {
    'Anchor': anchors[:27],
    'Positive': positives[:27],
    'Negative': negatives
}
df = pd.DataFrame(data)
df

,Anchor,Positive,Negative
0,img2.jpg,img1.jpg,img18.jpg
1,img4.jpg,img1.jpg,img14.jpg
2,img5.jpg,img1.jpg,img16.jpg
3,img6.jpg,img1.jpg,img15.jpg
4,img7.jpg,img1.jpg,img17.jpg
5,img10.jpg,img1.jpg,img3.jpg
6,img11.jpg,img1.jpg,img18.jpg
7,img4.jpg,img2.jpg,img23.jpg
8,img5.jpg,img2.jpg,img22.jpg
9,img6.jpg,img2.jpg,img1.jpg


## Evaluation dataset

In [7]:
# create positive images
positives = []

for key, values in idendities.items():
    
    #print(key)
    for i in range(0, len(values)-1):
        for j in range(i+1, len(values)):
            positive = []
            positive.append(values[i])
            positive.append(values[j])
            positives.append(positive)

In [66]:
positives = pd.DataFrame(positives, columns = ["file_x", "file_y"])
positives["Decision"] = 1

In [67]:
# create negative images
samples_list = list(idendities.values())
negatives = []

for i in range(0, len(idendities) - 1):
    for j in range(i+1, len(idendities)):
        cross_product = itertools.product(samples_list[i], samples_list[j])
        cross_product = list(cross_product)
        
        for cross_sample in cross_product:
            negative = []
            negative.append(cross_sample[0])
            negative.append(cross_sample[1])
            negatives.append(negative)

In [68]:
negatives = pd.DataFrame(negatives, columns = ["file_x", "file_y"])
negatives["Decision"] = 0

In [69]:
# Merge positive and negative images
eval = pd.concat([positives, negatives]).reset_index(drop = True)
eval.head()

,file_x,file_y,Decision
0,img1.jpg,img2.jpg,1
1,img1.jpg,img4.jpg,1
2,img1.jpg,img5.jpg,1
3,img1.jpg,img6.jpg,1
4,img1.jpg,img7.jpg,1


# Modelling

## Siamese Network

In [12]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 512, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        self.fc = nn.Sequential(
            nn.Linear(8192, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )

    def forward_one(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)  # Flatten the output
        output = self.fc(output)
        return output

    def forward(self, input):
        output = self.forward_one(input)
        return output

## triplet loss

In [13]:
class TripletLoss(nn.Module):
    def __init__(self, margin=0.2):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_dist = (anchor - positive).pow(2).sum(1)
        neg_dist = (anchor - negative).pow(2).sum(1)
        loss = torch.clamp(self.margin + pos_dist - neg_dist, min=0.0)
        return loss.mean()


## siamese dataset

In [14]:
class SiameseDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __getitem__(self, index):
        image1, image2,image3 = self.data.iloc[index]
        image1 = os.path.join ("./images",image1)
        image1 = cv2.imread(image1)
        image1 = self.transform(image1)
        image2 = os.path.join ("./images",image2)
        image2 = cv2.imread(image2)
        image2 = self.transform(image2)
        image3 = os.path.join ("./images",image3)
        image3 = cv2.imread(image3)
        image3 = self.transform(image3)
        return image1, image2, image3

    def __len__(self):
        return len(self.data)

In [15]:
class EvalDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __getitem__(self, index):
        image1, image2,label = self.data.iloc[index]
        image1 = os.path.join ("./images",image1)
        image1 = cv2.imread(image1)
        image1 = self.transform(image1)
        image2 = os.path.join ("./images",image2)
        image2 = cv2.imread(image2)
        image2 = self.transform(image2)
        return image1, image2, label

    def __len__(self):
        return len(self.data)

## Training phase

In [16]:
from tqdm import tqdm

# Data transformation
transform = transforms.Compose([transforms.ToPILImage(),transforms.Grayscale(num_output_channels=1),transforms.Resize((128, 128)), transforms.ToTensor()])

# Create a custom Siamese dataset
siamese_dataset = SiameseDataset(df, transform=transform)
# Data loader
train_loader = DataLoader(siamese_dataset, shuffle=True, batch_size=64)

# Initialize the Siamese network and triplet loss
net = SiameseNetwork()
triplet_loss = TripletLoss()

# Initialize the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.0001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader,0):
        input1, input2, input3 = data
        optimizer.zero_grad()
        anchor = net(input1)
        positive = net(input2)
        negative = net(input3)
        loss = triplet_loss(anchor, positive, negative)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print("Training finished.")


Epoch 1, Loss: 0.19974325597286224
Epoch 2, Loss: 0.19304609298706055
Epoch 3, Loss: 0.1531039923429489
Epoch 4, Loss: 0.09531861543655396
Epoch 5, Loss: 0.08292731642723083
Epoch 6, Loss: 0.09543445706367493
Epoch 7, Loss: 0.08568125218153
Epoch 8, Loss: 0.0644928440451622
Epoch 9, Loss: 0.047857142984867096
Epoch 10, Loss: 0.03858476132154465
Epoch 11, Loss: 0.03144490346312523
Epoch 12, Loss: 0.02350170910358429
Epoch 13, Loss: 0.01514359936118126
Epoch 14, Loss: 0.013040593825280666
Epoch 15, Loss: 0.010918752290308475
Epoch 16, Loss: 0.00814866740256548
Epoch 17, Loss: 0.008045352064073086
Epoch 18, Loss: 0.008191747590899467
Epoch 19, Loss: 0.008096970617771149
Epoch 20, Loss: 0.007875755429267883
Training finished.


In [17]:
# Saving models weights
torch.save(net, './Siamese-model.pt')

## Evaluation phase

In [70]:
# Create a evaluation dataset
eval_dataset = EvalDataset(eval, transform=transform)
# Data loader
eval_loader = DataLoader(eval_dataset, shuffle=True, batch_size=64)

# Set your model to evaluation mode
net.eval()

# Lists to store predictions and ground truth labels
predictions = []
labels = []
threshold = 0.5
with torch.no_grad():
    for batch in eval_loader:
        input1, input2, label = batch
        # Forward pass to get embeddings
        embedding1 = net(input1)
        embedding2 = net(input2)
        # Compute similarity scores between embeddings
        similarity_scores = torch.pairwise_distance(embedding1, embedding2)

        # Predict using a threshold
        predictions.extend([1 if score.item() < threshold else 0 for score in similarity_scores])
        labels.extend(label)



In [61]:
len(ground_truth)

351

In [72]:
def Evaluation (ground_truth, prediction):
    cm = confusion_matrix(ground_truth,prediction)
    true_positive = cm[0,0]
    false_positive = cm[0, 1]
    true_negative = cm[1, 1]
    false_negative = cm[1, 0]
    accuracy = accuracy_score(ground_truth, prediction)

    # Calculate precision, recall, and F1-score
    precision = precision_score(ground_truth, prediction)
    recall = recall_score(ground_truth, prediction)
    f1 = f1_score(ground_truth,prediction)
    # Calculate FAR (False Acceptance Rate)
    far = false_positive / (false_positive + true_negative)

    # Calculate FRR (False Rejection Rate)
    frr = false_negative / (false_negative + true_positive)

    # Calculate EER (Equal Error Rate)
    eer = 0.5 * (far + frr)
    return accuracy,precision,recall,f1,far,frr,eer

In [73]:
accuracy,precision,recall,f1,far,frr,eer = Evaluation (labels, predictions)
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1_score: ",f1)
print("FAR: ",far)
print("FRR: ",frr)
print("EER: ",eer)

Accuracy:  0.8717948717948718
Precision:  0.38461538461538464
Recall:  0.11904761904761904
F1_score:  0.18181818181818182
FAR:  0.6153846153846154
FRR:  0.10946745562130178
EER:  0.3624260355029586
